# MIL (Multiple Instance Learning) Tutorial

This tutorial demonstrates how to use Multiple Instance Learning (MIL) models with Slideflow for digital pathology tasks. 

## Setting up the environment

First, let's import the necessary libraries and set up our project.

In [1]:
import os
import slideflow as sf
from slideflow.mil import mil_config

# Set the backend to PyTorch
os.environ['SF_BACKEND'] = 'torch'

# Initialize the project
project_root = '/mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT'
P = sf.Project(root=project_root)

/home/mattsacco/miniconda3/envs/sf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Save features bags as torch tensors

When generating feature bags, make sure to save them as torch tensors. 
I commented out the code because the features should already be saved.

In [2]:
# extractor = 'ctranspath'
# weights_path = f'/mnt/labshare/MODELS/{extractor}/pytorch_model.bin'
# dataset = P.dataset(tile_px=299, tile_um=302)
# extractor_model = sf.model.build_feature_extractor(extractor, center_crop=True)
# features = sf.DatasetFeatures(extractor_model, dataset=dataset, normalizer='reinhard')
# features.to_torch(project_root + f'/features/{extractor}/torch')

## Splitting Data for Training and Validation

There are different strategies for splitting data between training and validation sets. Here are some common approaches:

In [3]:

# Option 1: Using filters
train_dataset = P.dataset(tile_px=299, tile_um=302, filters={'dataset': 'train'})
val_dataset = P.dataset(tile_px=299, tile_um=302, filters={'dataset': 'val'})

# Option 2: Using k-fold cross-validation
dataset = P.dataset(tile_px=299, tile_um=302)
splits = dataset.kfold_split(k=2, splits='./splits.json')

/home/mattsacco/miniconda3/envs/sf/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/mattsacco/miniconda3/envs/sf/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/mattsacco/miniconda3/envs/sf/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[10:38:31] INFO     Using k-fold validation split detected at ./splits.json (ID: 0)

           INFO     Using k-fold validation split detected at ./splits.json (ID: 0)

## Training a MIL Model

Now, let's train a MIL model using the Attention MIL architecture. 

First using the k-fold cross-validation approach then using the train and val datasets.


In [4]:
extractor = 'ctranspath'
model = 'attention_mil'

# k-fold cross-validation training
config = mil_config(model, bag_size=4, batch_size=2, epochs=2)
# FIXME: here bag_size, batch_size, and epochs are hardcoded to low values to 
#        make the tutorial run fast. Do not use these values.

for i, (train, val) in enumerate(splits):
    P.train_mil(
        config=config,
        exp_label=f'{extractor}_{model}_fold{i}',
        outcomes='cohort',
        train_dataset=train,
        val_dataset=val,
        bags=project_root + f'/features/{extractor}/torch'
    )

# FIXME: if you run this notebook you'll save many mil models at TEST_PROJECT/mil
#        please remove them so we don't accumulate too many models in the project.

           INFO     Training FastAI MIL model with config:

           INFO     TrainerConfig(                                                                                 
                      lr=None                                                                                      
                      wd=1e-05                                                                                     
                      bag_size=4                                                                                   
                      max_val_bag_size=None                                                                        
                      fit_one_cycle=True                                                                           
                      epochs=2                                                                                     
                      batch_size=2                                                                                 
                      drop_last=True                                                                               
                      save_monitor='valid_loss'                                                                    
                      weighted_loss=True                                                                           
                      model='attention_mil'                                                                        
                      model_kwargs=None                                                                            
                      loss='cross_entropy'                                                                         
                      use_lens=True                                                                                
                    )

           INFO     Training dataset: 5 merged bags (from 5 possible slides)

           INFO     Validation dataset: 5 merged bags (from 5 possible slides)

           INFO     Building model Attention_MIL (n_in=768, n_out=2)

...

[10:39:14] INFO     slide-level AUC (cat # 0): 0.333 AP: 0.450 (opt. threshold: 0.457)

           INFO     slide-level AUC (cat # 1): 0.333 AP: 0.589 (opt. threshold: 0.574)

           INFO     Category 0 acc: 0.0% (0/2)

           INFO     Category 1 acc: 100.0% (3/3)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00017-ctranspath_attention_mil_fold1/atte
                    ntion/TCGA-21-5782-01Z-00-DX1.E0C856C0-9055-4437-9393-695D699E4F57_att.npz

In [5]:
P.train_mil(
    config=config,
    exp_label=f'{extractor}_{model}',
    outcomes='cohort',
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    bags=project_root + f'/features/{extractor}/torch'
)

           INFO     Training FastAI MIL model with config:

           INFO     TrainerConfig(                                                                                 
                      lr=None                                                                                      
                      wd=1e-05                                                                                     
                      bag_size=4                                                                                   
                      max_val_bag_size=None                                                                        
                      fit_one_cycle=True                                                                           
                      epochs=2                                                                                     
                      batch_size=2                                                                                 
                      drop_last=True                                                                               
                      save_monitor='valid_loss'                                                                    
                      weighted_loss=True                                                                           
                      model='attention_mil'                                                                        
                      model_kwargs=None                                                                            
                      loss='cross_entropy'                                                                         
                      use_lens=True                                                                                
                    )

           INFO     Training dataset: 6 merged bags (from 6 possible slides)

           INFO     Validation dataset: 4 merged bags (from 4 possible slides)

           INFO     Building model Attention_MIL (n_in=768, n_out=2)

...

           INFO     slide-level AUC (cat # 0): 0.667 AP: 0.917 (opt. threshold: 0.410)

           INFO     slide-level AUC (cat # 1): 0.667 AP: 0.500 (opt. threshold: 0.591)

           INFO     Category 0 acc: 0.0% (0/3)

           INFO     Category 1 acc: 100.0% (1/1)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00018-ctranspath_attention_mil/attention/
                    TCGA-NC-A5HG-01Z-00-DX1.129E83E5-1C4D-4B31-8750-64D10A65267F_att.npz

## Evaluating a MIL Model

After training, it's important to evaluate the model's performance on a separate test set.

In [6]:
test_dataset = P.dataset(tile_px=299, tile_um=302, filters={'dataset': 'val'})

df = P.evaluate_mil(
    '/mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00000-ctranspath_attention_mil',
    outcomes='cohort',
    dataset=test_dataset,
    bags=project_root + f'/features/{extractor}/torch'
)

/home/mattsacco/miniconda3/envs/sf/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[10:39:31] INFO     Building model Attention_MIL (n_in=768, n_out=2)

           INFO     Loading model weights from                                                                     
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00000-ctranspath_attention_mil/models/bes
                    t_valid.pth

           INFO     Predictions saved to                                                                           
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil_eval/00001-attention_mil/predictions.parq
                    uet

           INFO     Validation metrics for outcome cohort:

           INFO     slide-level AUC (cat # 0): 1.000 AP: 1.000 (opt. threshold: 0.522)

           INFO     slide-level AUC (cat # 1): 1.000 AP: 1.000 (opt. threshold: 0.480)

           INFO     Category 0 acc: 100.0% (3/3)

           INFO     Category 1 acc: 0.0% (0/1)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil_eval/00001-attention_mil/attention/TCGA-N
                    C-A5HG-01Z-00-DX1.129E83E5-1C4D-4B31-8750-64D10A65267F_att.npz

## Different MIL Model Architectures

Slideflow supports various MIL architectures. Here are some examples:

- Attention MIL
- TransMIL
- CLAM-SB

Let's train models using different architectures:

In [7]:
models = ['transmil', 'attention_mil']

for model in models:
    config = mil_config(model, bag_size=4, batch_size=2, epochs=2)
    P.train_mil(
        config=config,
        exp_label=f'{extractor}_{model}',
        outcomes='cohort',
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        bags=project_root + f'/features/{extractor}/torch'
    )

           INFO     Training FastAI MIL model with config:

           INFO     TrainerConfig(                                                                                 
                      lr=None                                                                                      
                      wd=1e-05                                                                                     
                      bag_size=4                                                                                   
                      max_val_bag_size=None                                                                        
                      fit_one_cycle=True                                                                           
                      epochs=2                                                                                     
                      batch_size=2                                                                                 
                      drop_last=True                                                                               
                      save_monitor='valid_loss'                                                                    
                      weighted_loss=True                                                                           
                      model='transmil'                                                                             
                      model_kwargs=None                                                                            
                      loss='cross_entropy'                                                                         
                      use_lens=False                                                                               
                    )

           INFO     Training dataset: 6 merged bags (from 6 possible slides)

           INFO     Validation dataset: 4 merged bags (from 4 possible slides)

           INFO     Building model TransMIL (n_in=768, n_out=2)

...

           INFO     slide-level AUC (cat # 0): 0.667 AP: 0.917 (opt. threshold: 0.521)

           INFO     slide-level AUC (cat # 1): 0.667 AP: 0.500 (opt. threshold: 0.507)

           INFO     Category 0 acc: 66.7% (2/3)

           INFO     Category 1 acc: 100.0% (1/1)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00020-ctranspath_attention_mil/attention/
                    TCGA-NC-A5HG-01Z-00-DX1.129E83E5-1C4D-4B31-8750-64D10A65267F_att.npz

## Aggregation Level

The `aggregation_level` parameter determines how the bags are formed and how outcomes are assigned:

- If `aggregation_level` is 'slide' (default), the bags are comprised of tiles from one slide, and there will be an outcome for each slide.
- If `aggregation_level` is 'patient', the bags are comprised of tiles from all slides belonging to a patient, and there will be a single outcome for each patient. This only has an effect when all or some patients have multiple slides.



## Save Monitor

We can also change the `save_monitor` from the default 'loss' to 'roc_auc_score', in which case the model with the best AUC will be saved.

In [8]:
model = 'attention_mil'
config = mil_config(
    model, bag_size=4, batch_size=2, epochs=2,

    # aggregation_level='patient',
    save_monitor='roc_auc_score'
)

P.train_mil(
    config=config,
    exp_label=f'{extractor}_{model}_patient_auc',
    outcomes='cohort',
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    bags=project_root + f'/features/{extractor}/torch'
)

# Note: aggregation Level and Save Monitor have nothing to do with each other.
#       I just wanted to make the tutorial shorter.

           INFO     Training FastAI MIL model with config:

           INFO     TrainerConfig(                                                                                 
                      lr=None                                                                                      
                      wd=1e-05                                                                                     
                      bag_size=4                                                                                   
                      max_val_bag_size=None                                                                        
                      fit_one_cycle=True                                                                           
                      epochs=2                                                                                     
                      batch_size=2                                                                                 
                      drop_last=True                                                                               
                      save_monitor='roc_auc_score'                                                                 
                      weighted_loss=True                                                                           
                      model='attention_mil'                                                                        
                      model_kwargs=None                                                                            
                      loss='cross_entropy'                                                                         
                      use_lens=True                                                                                
                    )

           INFO     Training dataset: 6 merged bags (from 6 possible slides)

           INFO     Validation dataset: 4 merged bags (from 4 possible slides)

           INFO     Building model Attention_MIL (n_in=768, n_out=2)

...

           INFO     slide-level AUC (cat # 0): 1.000 AP: 1.000 (opt. threshold: 0.504)

           INFO     slide-level AUC (cat # 1): 1.000 AP: 1.000 (opt. threshold: 0.501)

           INFO     Category 0 acc: 100.0% (3/3)

           INFO     Category 1 acc: 100.0% (1/1)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00021-ctranspath_attention_mil_patient_au
                    c/attention/TCGA-NC-A5HG-01Z-00-DX1.129E83E5-1C4D-4B31-8750-64D10A65267F_att.npz

## Other Configurable Parameters

Aside from `epoch`, `bag_size`, `batch_size` - which we have been specifying to 
have the cell run fast and work on a small test dataset -
MIL models have many other parameters that can be adjusted to optimize performance.
Here's an example of how to modify some of these parameters:

In [10]:
config = mil_config(
    model,
    lr=1e-3,
    bag_size=4,
    epochs=3,
    batch_size=2
)

P.train_mil(
    config=config,
    exp_label=f'{extractor}_{model}_custom',
    outcomes='cohort',
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    bags=project_root + f'/features/{extractor}/torch'
)

[10:44:30] INFO     Training FastAI MIL model with config:

           INFO     TrainerConfig(                                                                                 
                      lr=0.001                                                                                     
                      wd=1e-05                                                                                     
                      bag_size=4                                                                                   
                      max_val_bag_size=None                                                                        
                      fit_one_cycle=True                                                                           
                      epochs=3                                                                                     
                      batch_size=2                                                                                 
                      drop_last=True                                                                               
                      save_monitor='valid_loss'                                                                    
                      weighted_loss=True                                                                           
                      model='attention_mil'                                                                        
                      model_kwargs=None                                                                            
                      loss='cross_entropy'                                                                         
                      use_lens=True                                                                                
                    )

           INFO     Training dataset: 6 merged bags (from 6 possible slides)

           INFO     Validation dataset: 4 merged bags (from 4 possible slides)

           INFO     Building model Attention_MIL (n_in=768, n_out=2)

...

           INFO     slide-level AUC (cat # 0): 0.667 AP: 0.917 (opt. threshold: 0.527)

           INFO     slide-level AUC (cat # 1): 0.667 AP: 0.500 (opt. threshold: 0.480)

           INFO     Category 0 acc: 100.0% (3/3)

           INFO     Category 1 acc: 0.0% (0/1)

           INFO     Attention scores exported to                                                                   
                    /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil/00023-ctranspath_attention_mil_custom/att
                    ention/TCGA-NC-A5HG-01Z-00-DX1.129E83E5-1C4D-4B31-8750-64D10A65267F_att.npz

## Clean up
remove all folder in `/mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil` except 
`00000-ctranspath_attention_mil`

In [12]:
import shutil

# Define the path to the folder
folder_path = '/mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil'

# List all directories in the folder
directories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
print(directories)

# Remove all directories except the one that has the name '00000-ctranspath_attention_mil'
for directory in directories:
    if directory != '00000-ctranspath_attention_mil':
        shutil.rmtree(os.path.join(folder_path, directory))

# print all folders in /mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil
print(os.listdir('/mnt/labshare/DL_OTHER/TEST_PROJECTS/TEST_PROJECT/mil')) 

['00022-ctranspath_attention_mil_custom', '00020-ctranspath_attention_mil', '00019-ctranspath_transmil', '00016-ctranspath_attention_mil_fold0', '00017-ctranspath_attention_mil_fold1', '00000-ctranspath_attention_mil', '00018-ctranspath_attention_mil', '00021-ctranspath_attention_mil_patient_auc', '00023-ctranspath_attention_mil_custom']
['00000-ctranspath_attention_mil']
